In [1]:
import tensorflow as tf
import numpy as np
import os


In [2]:
# IN and Out variables for data 
dimension = 60
fps = 11 #frames per second

x = tf.placeholder(tf.float32, [None, dimension, fps])  #Inputs
y = tf.placeholder(tf.float32, [None, 2])   #Labels

In [3]:
#convolution Layer - 1 
# [ filter_height, filter_width, in_channels, out_channels ]
weights_conv1 = tf.Variable( tf.truncated_normal( shape = [dimension, 3, 1, 128], stddev =0.01, dtype = tf.float32), dtype = tf.float32) 
biases_conv1 = tf.Variable( tf.constant(0.1, shape = [128]))

#reshape of input # -1 = dynamically caluclates the batch size , in_height, in_width, in_channels
input_x = tf.reshape(x, [-1, dimension, fps, 1])
#input_x = tf.to_double(input_x)
print(input_x)

#output
conv1 = tf.nn.conv2d( input_x, weights_conv1, strides=[1, 1, 1, 1], padding='VALID') 
conv1_out = tf.nn.relu( conv1 + biases_conv1 )

print(conv1_out)

Tensor("Reshape:0", shape=(?, 39, 100, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 1, 98, 128), dtype=float32)


In [4]:
#convolution layer - 2

weights_conv2 = tf.Variable( tf.truncated_normal( shape = [1, 3, 128, 128], stddev = 0.01 ))
biases_conv2 = tf.Variable( tf.constant(0.1, shape = [128] ))

#output
conv2 = tf.nn.conv2d( conv1_out, weights_conv2, strides = [1, 1, 1, 1], padding = 'VALID')
conv2_out = tf.nn.relu( conv2 + biases_conv2)

print(conv2_out)

Tensor("Relu_1:0", shape=(?, 1, 96, 128), dtype=float32)


In [5]:
#convolution layer - 3

weights_conv3 = tf.Variable( tf.truncated_normal(shape = [1, 3, 128, 128] , stddev = 0.01))
biases_conv3 = tf.Variable( tf.constant(0.1, shape = [128]))

#output
conv3 = tf.nn.conv2d( conv2_out, weights_conv3, strides = [1, 1, 1, 1], padding ='VALID')
conv3_out = tf.nn.relu( conv3 + biases_conv3)

print(conv3_out)

Tensor("Relu_2:0", shape=(?, 1, 94, 128), dtype=float32)


In [6]:
#Max pooling layer
#pool_out = tf.nn.max_pool(conv3_out, ksize = [1, 1, 2, 1], strides = [1, 1, 2, 1], padding = 'VALID')

weights_conv4 = tf.Variable( tf.truncated_normal(shape = [1, 2, 128, 128], stddev = 0.01))
biases_conv4 = tf.Variable( tf.constant(0.1, shape = [128]))

conv4 = tf.nn.conv2d( conv3_out, weights_conv4, strides = [1, 2, 2, 1], padding = "VALID")
conv4_out = tf.nn.relu( conv4 + biases_conv4 )

pool_out = conv4_out
print(pool_out)

Tensor("Relu_3:0", shape=(?, 1, 47, 128), dtype=float32)


In [7]:
#print(max_pool.shape)
red_dim = pool_out.shape[1] * pool_out.shape[2] * pool_out.shape[3]
#print(red_dim,"   ",47*128)

#Dense layer - 1
weights_dense1 = tf.Variable( tf.truncated_normal( shape = [int(red_dim), 256], stddev = 0.01 ))
biases_dense1 = tf.Variable( tf.constant( 0.1 , shape = [256]))

#Reshape of max_pool layer
input_dense = tf.reshape( pool_out, shape = [-1, int(red_dim)])

dense1 = tf.nn.relu( tf.matmul(input_dense, weights_dense1) + biases_dense1)

print(dense1)

Tensor("Relu_4:0", shape=(?, 256), dtype=float32)


In [8]:
#Adding dropout to dense -1 
keep_prob = tf.placeholder(dtype= tf.float32)
dense1_dropout = tf.nn.dropout( dense1, keep_prob )
print(dense1_dropout)

Tensor("dropout/mul:0", shape=(?, 256), dtype=float32)


In [9]:
# Dense layer - 2

weights_dense2 = tf.Variable( tf.truncated_normal( shape = [256, 256], stddev = 0.01))
biases_dense2 = tf.Variable( tf.constant( 0.1, shape =[256]))

#output
dense2 = tf.nn.relu( tf.matmul( dense1_dropout, weights_dense2 ) + biases_dense2 )

print(dense2)

Tensor("Relu_5:0", shape=(?, 256), dtype=float32)


In [10]:
#Dropout to dense - 2
dense2_dropout = tf.nn.dropout(dense2, keep_prob)

print( dense2)

Tensor("Relu_5:0", shape=(?, 256), dtype=float32)


In [11]:
#Dense layer - 3

weights_dense3 = tf.Variable( tf.truncated_normal( shape = [256, 256], stddev = 0.01))
biases_dense3 = tf.Variable( tf.constant( 0.1 , shape = [256]))

#output
dense3 = tf.nn.relu( tf.matmul( dense2_dropout, weights_dense3) + biases_dense3)

print(dense3)

Tensor("Relu_6:0", shape=(?, 256), dtype=float32)


In [12]:
# dropout to layer 3
dense3_dropout = tf.nn.dropout( dense3, keep_prob)

print(dense3_dropout)

Tensor("dropout_2/mul:0", shape=(?, 256), dtype=float32)


In [13]:
#Final output layer

weights_final = tf.Variable( tf.truncated_normal( shape = [256, 2], stddev = 0.01 ))
biases_final = tf.Variable( tf.constant( 0.1, shape =[2] ))


In [14]:
output = tf.matmul( dense3_dropout, weights_final ) + biases_final
output1 = tf.nn.softmax(output)

In [15]:
# losses and optimizers

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = output))
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal( tf.argmax(output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean( tf.cast( correct_predictions, tf.float32 ))

In [16]:
# matric for testing on development 

test_features = []
test_labels = []

test_path ="E:\\siva\\RLMS_feat\\CNN_input_testing_dev"

for folders in os.listdir(test_path):
    print(folders)
    features = np.load(os.path.join(test_path, folders, "feature_image.npy"))
    print(features.shape)
    labels = np.load(os.path.join(test_path, folders, "label_image.npy"))
    for i in range(labels.shape[0]):
        test_features.append(features[i])
        test_labels.append(labels[i])
    
test_features = np.array(test_features)
test_labels = np.array(test_labels)
print(test_features.shape)
print(test_labels.shape)


0
(5, 39, 100)
1
(3, 39, 100)
10
(5, 39, 100)
100
(4, 39, 100)
1000
(2, 39, 100)
1001
(2, 39, 100)
1002
(3, 39, 100)
1003
(2, 39, 100)
1004
(3, 39, 100)
1005
(3, 39, 100)
1006
(3, 39, 100)
1007
(4, 39, 100)
1008
(3, 39, 100)
1009
(3, 39, 100)
101
(2, 39, 100)
1010
(2, 39, 100)
1011
(4, 39, 100)
1012
(4, 39, 100)
1013
(4, 39, 100)
1014
(3, 39, 100)
1015
(3, 39, 100)
1016
(3, 39, 100)
1017
(3, 39, 100)
1018
(3, 39, 100)
1019
(3, 39, 100)
102
(3, 39, 100)
1020
(3, 39, 100)
1021
(3, 39, 100)
1022
(2, 39, 100)
1023
(3, 39, 100)
1024
(3, 39, 100)
1025
(2, 39, 100)
1026
(2, 39, 100)
1027
(5, 39, 100)
1028
(5, 39, 100)
1029
(3, 39, 100)
103
(3, 39, 100)
1030
(2, 39, 100)
1031
(3, 39, 100)
1032
(3, 39, 100)
1033
(3, 39, 100)
1034
(3, 39, 100)
1035
(4, 39, 100)
1036
(3, 39, 100)
1037
(2, 39, 100)
1038
(3, 39, 100)
1039
(2, 39, 100)
104
(4, 39, 100)
1040
(3, 39, 100)
1041
(4, 39, 100)
1042
(3, 39, 100)
1043
(2, 39, 100)
1044
(2, 39, 100)
1045
(3, 39, 100)
1046
(4, 39, 100)
1047
(4, 39, 100)
1048


(4, 39, 100)
1461
(4, 39, 100)
1462
(4, 39, 100)
1463
(3, 39, 100)
1464
(4, 39, 100)
1465
(5, 39, 100)
1466
(4, 39, 100)
1467
(3, 39, 100)
1468
(4, 39, 100)
1469
(2, 39, 100)
147
(3, 39, 100)
1470
(2, 39, 100)
1471
(3, 39, 100)
1472
(3, 39, 100)
1473
(3, 39, 100)
1474
(2, 39, 100)
1475
(2, 39, 100)
1476
(2, 39, 100)
1477
(3, 39, 100)
1478
(2, 39, 100)
1479
(3, 39, 100)
148
(3, 39, 100)
1480
(3, 39, 100)
1481
(3, 39, 100)
1482
(4, 39, 100)
1483
(3, 39, 100)
1484
(3, 39, 100)
1485
(2, 39, 100)
1486
(4, 39, 100)
1487
(3, 39, 100)
1488
(4, 39, 100)
1489
(4, 39, 100)
149
(4, 39, 100)
1490
(3, 39, 100)
1491
(3, 39, 100)
1492
(3, 39, 100)
1493
(3, 39, 100)
1494
(3, 39, 100)
1495
(3, 39, 100)
1496
(3, 39, 100)
1497
(2, 39, 100)
1498
(3, 39, 100)
1499
(3, 39, 100)
15
(3, 39, 100)
150
(2, 39, 100)
1500
(2, 39, 100)
1501
(2, 39, 100)
1502
(5, 39, 100)
1503
(5, 39, 100)
1504
(3, 39, 100)
1505
(2, 39, 100)
1506
(3, 39, 100)
1507
(3, 39, 100)
1508
(3, 39, 100)
1509
(3, 39, 100)
151
(3, 39, 100)
1510

397
(2, 39, 100)
398
(4, 39, 100)
399
(5, 39, 100)
4
(4, 39, 100)
40
(3, 39, 100)
400
(5, 39, 100)
401
(4, 39, 100)
402
(5, 39, 100)
403
(4, 39, 100)
404
(3, 39, 100)
405
(5, 39, 100)
406
(5, 39, 100)
407
(4, 39, 100)
408
(5, 39, 100)
409
(5, 39, 100)
41
(4, 39, 100)
410
(3, 39, 100)
411
(4, 39, 100)
412
(2, 39, 100)
413
(4, 39, 100)
414
(3, 39, 100)
415
(4, 39, 100)
416
(4, 39, 100)
417
(4, 39, 100)
418
(4, 39, 100)
419
(3, 39, 100)
42
(3, 39, 100)
420
(4, 39, 100)
421
(4, 39, 100)
422
(4, 39, 100)
423
(5, 39, 100)
424
(2, 39, 100)
425
(4, 39, 100)
426
(4, 39, 100)
427
(4, 39, 100)
428
(4, 39, 100)
429
(3, 39, 100)
43
(5, 39, 100)
430
(4, 39, 100)
431
(4, 39, 100)
432
(4, 39, 100)
433
(5, 39, 100)
434
(2, 39, 100)
435
(4, 39, 100)
436
(4, 39, 100)
437
(4, 39, 100)
438
(5, 39, 100)
439
(4, 39, 100)
44
(4, 39, 100)
440
(5, 39, 100)
441
(5, 39, 100)
442
(4, 39, 100)
443
(4, 39, 100)
444
(4, 39, 100)
445
(4, 39, 100)
446
(3, 39, 100)
447
(4, 39, 100)
448
(4, 39, 100)
449
(3, 39, 100)
45
(

862
(4, 39, 100)
863
(11, 39, 100)
864
(11, 39, 100)
865
(11, 39, 100)
866
(5, 39, 100)
867
(4, 39, 100)
868
(4, 39, 100)
869
(3, 39, 100)
87
(4, 39, 100)
870
(3, 39, 100)
871
(5, 39, 100)
872
(3, 39, 100)
873
(3, 39, 100)
874
(2, 39, 100)
875
(3, 39, 100)
876
(5, 39, 100)
877
(4, 39, 100)
878
(4, 39, 100)
879
(2, 39, 100)
88
(4, 39, 100)
880
(4, 39, 100)
881
(4, 39, 100)
882
(3, 39, 100)
883
(2, 39, 100)
884
(2, 39, 100)
885
(4, 39, 100)
886
(4, 39, 100)
887
(4, 39, 100)
888
(3, 39, 100)
889
(3, 39, 100)
89
(3, 39, 100)
890
(4, 39, 100)
891
(4, 39, 100)
892
(4, 39, 100)
893
(3, 39, 100)
894
(4, 39, 100)
895
(5, 39, 100)
896
(4, 39, 100)
897
(3, 39, 100)
898
(4, 39, 100)
899
(2, 39, 100)
9
(3, 39, 100)
90
(4, 39, 100)
900
(2, 39, 100)
901
(3, 39, 100)
902
(3, 39, 100)
903
(3, 39, 100)
904
(2, 39, 100)
905
(2, 39, 100)
906
(2, 39, 100)
907
(3, 39, 100)
908
(2, 39, 100)
909
(3, 39, 100)
91
(3, 39, 100)
910
(3, 39, 100)
911
(3, 39, 100)
912
(4, 39, 100)
913
(3, 39, 100)
914
(3, 39, 100)
9

In [17]:
#initializing tensors using sessoion
session = tf.InteractiveSession()
init = tf.global_variables_initializer()

In [6]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [45]:

from scipy.io import loadmat,savemat

scores = loadmat('E:\\Aditya\\Madhu Mam\\VESA-ESA-HT\\VESA-IFCC\\2\\dev_scores.mat')
print(scores)


dev_genuine_scores = scores['scores'][0:4995]
dev_spoof_scores = scores['scores'][4995:]

print(dev_genuine_scores.shape)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Thu Mar 28 20:16:03 2019', '__version__': '1.0', '__globals__': [], 'scores': array([[ 1.23176247],
       [ 1.49361322],
       [ 1.38973614],
       ..., 
       [-1.60276951],
       [-1.02082924],
       [-1.73461944]])}
(4995, 1)


In [46]:
cal_eer(dev_genuine_scores,dev_spoof_scores)

FRR   0.05325325325325325  FAR    0.05339554173146708    mid    [ 0.53491704]


(0.05325325325325325, array([ 0.53491704]))

In [47]:

scores = loadmat('E:\\Aditya\\Madhu Mam\\VESA-ESA-HT\\VESA-IFCC\\2\\eval_scores.mat')
print(scores)


eval_genuine_scores = scores['scores'][0:7042]
eval_spoof_scores = scores['scores'][7042:]

print(eval_genuine_scores.shape)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sun Mar 31 07:24:04 2019', '__version__': '1.0', '__globals__': [], 'scores': array([[ 1.12133625],
       [ 0.89047821],
       [ 0.94014617],
       ..., 
       [-1.70021899],
       [-1.05271614],
       [-1.80922062]])}
(7042, 1)


In [48]:
def cal_hter(scores_true, scores_false, mid):
    
    FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
    FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
    
    HTER = (FAR+FRR)/2
    return FAR, FRR, HTER

In [49]:
cal_hter(eval_genuine_scores,eval_spoof_scores,0.53491704)

(0.1153125, 0.027548991763703495, 0.07143074588185175)

In [33]:
a = 'G:\\Spoof Database\\ESA-IFCC\\Eval Scores'
b = os.listdir('G:\\Spoof Database\\ESA-IFCC\\Eval Scores')

for i in range(0,len(b)):
    k = loadmat(os.path.join(a,b[i]))[b[i][:-4]]
    print(k.shape)
    
   

(1000, 1)
(1000, 1)
(800, 1)
(1000, 1)
(1000, 1)
(800, 1)
(700, 1)
(700, 1)
(23700, 1)
(23700, 1)


In [34]:
import os
from scipy.io import loadmat,savemat
a = 'G:\\Spoof Database\\ESA-IFCC\\Eval Scores'
b = os.listdir('G:\\Spoof Database\\ESA-IFCC\\Eval Scores')
evl_scores = loadmat('G:\\Spoof Database\\ESA-IFCC\\scores_eval.mat')

scores = evl_scores['scores_eval']
gen_scores = scores[:7042]


# j = 0

for i in range(0,len(b)):
     
    
#     combine = []
#     gen = loadmat(os.path.join(a,b[j]))[b[j][:-4]]
# #     print(gen.shape)
#     for k in range(0,len(b)):
#         if(j != k):
    temp = loadmat(os.path.join(a,b[i]))[b[i][:-4]]
#             combine.extend((temp))
#     j = j+1
#     combine.extend(gen_scores)
    print(b[i])
    print(cal_hter(gen_scores,temp,0.58273278))
    

scores_RL.mat
(0.402, 0.045299630786708324, 0.22364981539335418)
scores_RLHQ.mat
(0.316, 0.045299630786708324, 0.18064981539335417)
scores_RLP3.mat
(0.94875, 0.045299630786708324, 0.49702481539335414)
scores_RP1.mat
(0.939, 0.045299630786708324, 0.4921498153933541)
scores_RP2.mat
(0.885, 0.045299630786708324, 0.46514981539335415)
scores_RP2P3.mat
(0.9775, 0.045299630786708324, 0.5113998153933542)
scores_SSPA.mat
(0.33285714285714285, 0.045299630786708324, 0.1890783868219256)
scores_SSPAHQ.mat
(0.22142857142857142, 0.045299630786708324, 0.13336410110763988)
scores_VCPA.mat
(0.024641350210970463, 0.045299630786708324, 0.03497049049883939)
scores_VCPAHQ.mat
(0.03130801687763713, 0.045299630786708324, 0.03830382383217273)


53400


In [100]:
len(combine)

53400

In [89]:
a = [1, 2, 3]
b = [12,12]

a.extend(b)
a

[1, 2, 3, 12, 12]

In [72]:
a = eval_scores['scores_eval']
a[1]
len(a)

61442

In [58]:
cal_hter(eval_genuine_scores,ev_spf_scores,3.32410111)

(0.005105485232067511, 0.02357284862255041, 0.014339166927308961)

In [19]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"

    for folders in os.listdir(test_path):
        features = np.load(os.path.join(test_path, folders, "feature_image.npy"))
        labels = np.load(os.path.join(test_path, folders, "label_image.npy"))
        patches = labels.shape[0]
        patch_prob = session.run(output1, feed_dict = {x:features, keep_prob : 1.0})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [20]:
#Training and testing accuracy
saver = tf.train.Saver(max_to_keep=None)
session.run(init)

features_path = "E:\\siva\\RLMS_feat\\CNN_input_train_data_overlap"
development_path = "E:\\siva\\RLMS_feat\\CNN_input_testing_dev"
evaluation_path = "E:\\siva\\RLMS_feat\\CNN_input_testing_eval"
model_path = "E:\\siva\\RLMS_feat\\dev_only\\4_conv_overlap\\Models\\"
    
dev_sp_path = "E:\\siva\\RLMS_feat\\dev_only\\4_conv_overlap\\dev"
eva_sp_path = "E:\\siva\\RLMS_feat\\dev_only\\4_conv_overlap\\eval" 

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    
for i in range(50):
    for j in range(0,batches):  #number of batches
        filename = "batch_"+str(j)+"_features.npy"
        labelsname= "batch_"+str(j)+"_labels.npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))
       
        #print("before  ",str(j),"   ",accuracy.eval(feed_dict = {x:features, y:labels, keep_prob:1.0}))
        train_step.run(feed_dict={ x:features, y:labels, keep_prob:0.5})
        #print("after   ",str(j),"   ",accuracy.eval(feed_dict = {x:features, y:labels, keep_prob:1.0}))
        
   
    test_accuracy = accuracy.eval(feed_dict = { x:test_features, y:test_labels, keep_prob:1.0 })
    print("iteration   ",(i+1),"   dev_accuracy    ",test_accuracy)    
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    eer_eval = get_EER(evaluation_path, (i+1), eva_sp_path)
    print(" evaluat eer  ",eer_eval)
    print()
    save_path = saver.save(session, model_path +str(i))
    

iteration    1    dev_accuracy     0.556519
FRR   0.20657894736842106  FAR    0.2073684210526316    mid    -9.63358571528
 develop eer   0.20657894736842106
FRR   0.28582434514637906  FAR    0.2860592938041306    mid    -10.8169162702
 evaluat eer   0.28582434514637906

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\0 is not in all_model_checkpoint_paths. Manually adding it.
iteration    2    dev_accuracy     0.555185
FRR   0.22105263157894736  FAR    0.22105263157894736    mid    -10.8070247168
 develop eer   0.22105263157894736
FRR   0.27426810477657937  FAR    0.2742338441039307    mid    -12.0762897187
 evaluat eer   0.27426810477657937

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\1 is not in all_model_checkpoint_paths. Manually adding it.
iteration    3    dev_accuracy     0.565689
FRR   0.16710526315789473  FAR    0.16736842105263158    mid    -10.9100625667
 develop eer   0.16710526315789473
FRR   0.2465331278890601  FAR    0.24691872085276

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\20 is not in all_model_checkpoint_paths. Manually adding it.
iteration    22    dev_accuracy     0.735912
FRR   0.10657894736842105  FAR    0.10736842105263159    mid    -12.5484546125
 develop eer   0.10657894736842105
FRR   0.18412942989214176  FAR    0.18437708194536975    mid    -11.9170272959
 evaluat eer   0.18412942989214176

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\21 is not in all_model_checkpoint_paths. Manually adding it.
iteration    23    dev_accuracy     0.736746
FRR   0.10921052631578948  FAR    0.10842105263157895    mid    -13.9503460724
 develop eer   0.10921052631578948
FRR   0.18335901386748846  FAR    0.18337774816788807    mid    -13.8413237434
 evaluat eer   0.18335901386748846

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\22 is not in all_model_checkpoint_paths. Manually adding it.
iteration    24    dev_accuracy     0.744415
FRR   0.11315789473684211  FA

FRR   0.17257318952234207  FAR    0.17230179880079946    mid    -11.7430512838
 evaluat eer   0.17257318952234207

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\41 is not in all_model_checkpoint_paths. Manually adding it.
iteration    43    dev_accuracy     0.777759
FRR   0.11315789473684211  FAR    0.11368421052631579    mid    -12.7300550938
 develop eer   0.11315789473684211
FRR   0.17334360554699538  FAR    0.17371752165223184    mid    -11.0782492418
 evaluat eer   0.17334360554699538

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\42 is not in all_model_checkpoint_paths. Manually adding it.
iteration    44    dev_accuracy     0.783595
FRR   0.11710526315789474  FAR    0.11789473684210526    mid    -13.8716479223
 develop eer   0.11710526315789474
FRR   0.17411402157164868  FAR    0.17438374417055297    mid    -11.0617576839
 evaluat eer   0.17411402157164868

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\43 is not in all_mod

In [21]:
for i in range(50,100):
    for j in range(0,batches):  #number of batches
        filename = "batch_"+str(j)+"_features.npy"
        labelsname= "batch_"+str(j)+"_labels.npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))
    
        #print("before  ",str(j),"   ",accuracy.eval(feed_dict = {x:features, y:labels, keep_prob:1.0}))
        train_step.run(feed_dict={ x:features, y:labels, keep_prob:0.7})
        #print("after   ",str(j),"   ",accuracy.eval(feed_dict = {x:features, y:labels, keep_prob:1.0}))
        
   
    test_accuracy = accuracy.eval(feed_dict = { x:test_features, y:test_labels, keep_prob:1.0 })
    print("iteration   ",(i+1),"   dev_accuracy    ",test_accuracy)    
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    eer_eval = get_EER(evaluation_path, (i+1), eva_sp_path)
    print(" evaluat eer  ",eer_eval)
    print()
    save_path = saver.save(session, model_path +str(i))

iteration    51    dev_accuracy     0.779093
FRR   0.11710526315789474  FAR    0.1168421052631579    mid    -13.7799681524
 develop eer   0.11710526315789474
FRR   0.17026194144838214  FAR    0.1703031312458361    mid    -11.2355296639
 evaluat eer   0.17026194144838214

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\50 is not in all_model_checkpoint_paths. Manually adding it.
iteration    52    dev_accuracy     0.775258
FRR   0.11447368421052631  FAR    0.11368421052631579    mid    -16.4118795816
 develop eer   0.11447368421052631
FRR   0.1687211093990755  FAR    0.1692205196535643    mid    -14.90006423
 evaluat eer   0.1687211093990755

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\51 is not in all_model_checkpoint_paths. Manually adding it.
iteration    53    dev_accuracy     0.8001
FRR   0.10657894736842105  FAR    0.10736842105263159    mid    -13.777646857
 develop eer   0.10657894736842105
FRR   0.16640986132511557  FAR    0.166472351765489

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\70 is not in all_model_checkpoint_paths. Manually adding it.
iteration    72    dev_accuracy     0.808103
FRR   0.10526315789473684  FAR    0.10631578947368421    mid    -12.7375351489
 develop eer   0.10526315789473684
FRR   0.16640986132511557  FAR    0.16663890739506995    mid    -7.99465804067
 evaluat eer   0.16640986132511557

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\71 is not in all_model_checkpoint_paths. Manually adding it.
iteration    73    dev_accuracy     0.773424
FRR   0.12368421052631579  FAR    0.12421052631578948    mid    -16.8782687433
 develop eer   0.12368421052631579
FRR   0.17334360554699538  FAR    0.17388407728181213    mid    -12.2741217811
 evaluat eer   0.17334360554699538

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\72 is not in all_model_checkpoint_paths. Manually adding it.
iteration    74    dev_accuracy     0.771757
FRR   0.11578947368421053  FA

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\91 is not in all_model_checkpoint_paths. Manually adding it.
iteration    93    dev_accuracy     0.823441
FRR   0.10263157894736842  FAR    0.1031578947368421    mid    -4.52036712086
 develop eer   0.10263157894736842
FRR   0.162557781201849  FAR    0.16272485009993337    mid    0.579754754914
 evaluat eer   0.162557781201849

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\92 is not in all_model_checkpoint_paths. Manually adding it.
iteration    94    dev_accuracy     0.787262
FRR   0.1118421052631579  FAR    0.11263157894736842    mid    -20.3466015102
 develop eer   0.1118421052631579
FRR   0.16718027734976887  FAR    0.1675549633577615    mid    -14.5980366115
 evaluat eer   0.16718027734976887

INFO:tensorflow:E:\siva\RLMS_feat\dev_only\4_conv_overlap\Models\93 is not in all_model_checkpoint_paths. Manually adding it.
iteration    95    dev_accuracy     0.809103
FRR   0.11052631578947368  FAR    0.1

In [12]:
import pandas as pd
file_path = 'E:\\Aditya\\Madhu Mam\\'
xl = pd.ExcelFile(file_path + 'results.xlsx')
xl.sheet_names

['Sheet1']

In [13]:
df = xl.parse('Sheet1')
# df.iloc[1:20, 1:5:1]

In [14]:
df.columns

Index(['Wav File', 'Label', 'IFCC_PRE', 'Unnamed: 3', 'IFCC_CMN', 'Unnamed: 5',
       'MGDCC', 'Unnamed: 7', 'LFCC', 'Unnamed: 9', 'CQCC'],
      dtype='object')

In [15]:
a = df['IFCC_PRE']
b = df['IFCC_CMN']
c = df['MGDCC']
d = df['LFCC']
e = df['CQCC']

label = df['Label']

In [14]:
label[132951]

'N'

In [15]:
for i in range(120000,124000):
    if(a[i]<0 and b[i]<0):
        label[i] = 'S'
    else:
        label[i] = ''

C:\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
df.head()

,Wav File,Label,IFCC_PRE,Unnamed: 3,IFCC_CMN,Unnamed: 5,MGDCC,Unnamed: 7,LFCC,Unnamed: 9,CQCC
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,N,1.371360,NaN,5.379075,NaN,-0.167523,NaN,1.981235,NaN,2.046415
2,2.0,S,-1.539372,NaN,0.533746,NaN,-0.352773,NaN,-1.207913,NaN,-1.842327
3,3.0,NaN,0.193455,NaN,1.629714,NaN,-0.787703,NaN,-0.160936,NaN,1.211040
4,4.0,S,-11.656062,NaN,-5.402439,NaN,-3.156255,NaN,-2.308487,NaN,-5.925815


In [18]:
export_excel = df.to_excel(r'E:\Aditya\Madhu Mam\results.xlsx', index = None, header = True)

In [16]:

# len(label)

count = 0
import math

for i in range(1,len(label)):
    if(label[i] == 'S' or label[i] == 'N'):
        count = count+1
print(count)

99289


In [7]:
137458 - 98560

38898

In [9]:
38898/137458

0.2829809832821662